In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
image_path = "./dataset_f+r20/images.npy"
emotion_multi_path = "./dataset_f+r20/emotions_multi.npy"
emotion_single_path = "./dataset_f+r20/emotions_single.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = layers.Rescaling(1./127.5, offset=-1)(images)
images = tf.image.grayscale_to_rgb(images)

emotions_multi = np.load(emotion_multi_path)
emotions_single = np.load(emotion_single_path)
emotions_multi = tf.convert_to_tensor(emotions_multi)
emotions_single = tf.convert_to_tensor(emotions_single)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions_multi = emotions_multi[:training_size]
test_emotions_multi = emotions_multi[training_size:]
training_emotions_single = emotions_single[:training_size]
test_emotions_single = emotions_single[training_size:]

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
# multi = 1: multi-label image
# multi = 0: single-label image
def train(model, learning_rate, loss, num_epochs, batch_size, multi):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    if multi == 1:
        history = model.fit(x=training_images,
                            y=training_emotions_multi,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            validation_data=(test_images, test_emotions_multi))
    if multi == 0:
        history = model.fit(x=training_images,
                            y=training_emotions_single,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            validation_data=(test_images, test_emotions_single))
    del model
    gc.collect()
    return history

In [ ]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 1e-4
num_epochs = 40
batch_size = 32
val_accs = []
for i in range(4):
    model = create_model()
    if i == 0:
        history = train(model, learning_rate, losses.MeanSquaredError(), num_epochs, batch_size, 1)
        history_save_path = './results/multi-mse.txt'
    if i == 1:
        history = train(model, learning_rate, losses.CategoricalCrossentropy(), num_epochs, batch_size, 1)
        history_save_path = './results/multi-cce.txt'
    if i == 2:
        history = train(model, learning_rate, losses.MeanSquaredError(), num_epochs, batch_size, 0)
        history_save_path = './results/single-mse.txt'
    if i == 3:
        history = train(model, learning_rate, losses.CategoricalCrossentropy(), num_epochs, batch_size, 0)
        history_save_path = './results/single-cce.txt'
    val_accs.append(history.history['val_model_acc'])
    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
# model comparison
epochs = range(1, 1+num_epochs)
fig = plt.figure()
plot_labels = ['multi-mse', 'multi-cce', 'single-mse', 'single-cce']
for i in range(4):
    plt.plot(epochs, val_accs[i], label = plot_labels[i])
plt.legend()
plt.show()

# save plot
fig.savefig('./new-results/input-type-and-loss-func.png')